In [158]:
import requests

from collections import Counter
import operator
import nltk
from bs4 import BeautifulSoup

import pickle
import xmltodict, json
import time

import numpy as np
import pandas as pd

import re

In [159]:
grkey = pickle.load( open( "grkey.p", "rb" ) )

In [160]:
# Main page thread: 18661546
main_page = requests.request('GET', 'https://hacker-news.firebaseio.com/v0/item/18661546.json').json()

In [161]:
text = []
i  = 0
kids = main_page["kids"]
kidspast = main_page["kids"]
while (kids != []) and (len(kidspast)<main_page['descendants']-1):
    kidsnew = []
    for kid in kids:
        i += 1
        item = requests.request('GET', 'https://hacker-news.firebaseio.com/v0/item/{kid}.json'.format(kid=kid)).json()
        try:
            soup = BeautifulSoup(item['text'])
            text.append(soup.findAll(text=True))
        except:
            pass
        try:
            kidsnew += item['kids']
        except:
            pass
    kidspast += list(set(kidsnew))
    kidspast = list(set(kidspast))
    kids = [k for k in list(set(kidsnew)) if k not in kidspast]
    
flat_list = [item for sublist in text for item in sublist]

In [162]:
pickle.dump(text, open( "text.p", "wb" ))

In [163]:
text = pickle.load( open( "text.p", "rb" ) )

In [164]:
text_clean = [re.sub(r"[^a-zA-Z0-9]+", ' ', k)  for t in flat_list for k in t.split("\n")]

In [165]:
countsb = Counter()
countst = Counter()
countsq = Counter()
countsc = Counter()

words = re.compile(r'\w+')
for t in text_clean:
    w = words.findall(t.lower())
    countsb.update(zip(w,w[1:]))
    countst.update(zip(w,w[1:],w[2:]))
    countsq.update(zip(w,w[1:],w[2:],w[3:]))
    countsc.update(zip(w,w[1:],w[2:],w[3:],w[4:]))

In [166]:
results = {}

# save the results
results[1] = sorted(
    countsb.items(),
    key=operator.itemgetter(1),
    reverse=True
)

results[2] = sorted(
    countst.items(),
    key=operator.itemgetter(1),
    reverse=True
)

results[3] = sorted(
    countsq.items(),
    key=operator.itemgetter(1),
    reverse=True
)

results[4] = sorted(
    countsc.items(),
    key=operator.itemgetter(1),
    reverse=True
)

In [167]:
# save the results
bigrams = sorted(
    countsb.items(),
    key=operator.itemgetter(1),
    reverse=True
)

In [168]:
bigrams[:10]

[((u'of', u'the'), 147),
 ((u'in', u'the'), 76),
 ((u'it', u's'), 67),
 ((u'this', u'book'), 52),
 ((u'this', u'year'), 49),
 ((u'if', u'you'), 45),
 ((u'and', u'the'), 44),
 ((u'i', u've'), 44),
 ((u'to', u'the'), 40),
 ((u'i', u'read'), 37)]

In [169]:
stops = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
         "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", 
         "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", 
         "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
         "have", "has", "had", "having", "do", "does", "did", "doing", "and", "an", "a", "the",  "but", 
         "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", 
         "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", 
         "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
         "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", 
         "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", 
         "s", "t", "can", "will", "just", "don", "should", "now", 'https', "www", "goodreads", "didn", "isn", "doesn",
        "recommend"]

In [170]:
df = {}
for k in [1,2,3,4]:
    names = []
    counts = []
    for c in results[k]:
        lst = list(c[0])
        lst = [w for w in lst if ((w not in stops) and w.isalpha())]
        if c[1]>2 & len(lst)>0:
            names.append(' '.join(c[0]))
            counts.append(c[1])
    df[k] = pd.DataFrame(data={'names':names, 'counts':counts})
    print(len(df[k]))

143
114
56
20


In [181]:
for jj in [1,2,3,4]:
    df[jj]['gr_title'] = None
    df[jj]['gr_author'] = None
    
    for it in df[jj].iterrows():
        name = it[1]['names']
        res = requests.get("https://www.goodreads.com/search/index.xml" , params={"key": grkey, "q":name})
        xpars = xmltodict.parse(res.text)
        json1 = json.dumps(xpars)
        d = json.loads(json1) 

        if d['GoodreadsResponse']['search']['results'] is None:
            title = None
            author = None
        elif type(d['GoodreadsResponse']['search']['results']['work']) == dict:
            title = d['GoodreadsResponse']['search']['results']['work']['best_book']['title']
            author = d['GoodreadsResponse']['search']['results']['work']['best_book']['author']['name']
        else:
            lst = d['GoodreadsResponse']['search']['results']['work']
            ys = []
            for j in range(len(lst)): 
                if '#text' in lst[j]['original_publication_year'].keys():
                    ys.append(int(lst[j]['original_publication_year']['#text']))
                else:
                    ys.append(1)
            j = np.argmax(ys)    
            title = lst[j]['best_book']['title']
            author = lst[j]['best_book']['author']['name']

        df[jj]['gr_title'].loc[it[0]] = title
        df[jj]['gr_author'].loc[it[0]] = author
        time.sleep(1)

In [182]:
dff = pd.DataFrame.copy(df[1])
dff = dff.append(df[2])
dff = dff.append(df[3])
dff = dff.append(df[4])

In [183]:
dff = dff.drop_duplicates(subset=['gr_title', 'gr_author'])
dff = dff[dff.gr_title.notnull()]

In [184]:
books = []
titles = []
authors = []

for it in dff.iterrows():
    books.append(it[1]['gr_title'].split(':')[0].split('(')[0])    
    titles.append(it[1]['gr_title'])
    authors.append(it[1]['gr_author'])    

In [185]:
conts = []
for b in books:
    count = 0
    for f in flat_list:
        if f.lower().find(b.lower()) >= 0:
            count += 1
    conts.append(count)
print(sum(conts))

114


In [186]:
dfp = pd.DataFrame(data={'books':books, 'counts':conts, 'title':titles, 'author':authors})
dfp = dfp[['books', 'counts', 'title', 'author']]

In [188]:
dfp[dfp['counts']>2].sort_values('counts', ascending=False).reset_index(drop=True)

,books,counts,title,author
0,Bad Blood,12,Bad Blood: Secrets and Lies in a Silicon Valle...,John Carreyrou
1,Why We Sleep,7,Why We Sleep: Unlocking the Power of Sleep and...,Matthew Walker
2,Magi,7,Magi: Uncovering the Secret Society That Read ...,Adrian Geoffrey Gilbert
3,Shoe Dog,6,Shoe Dog: A Memoir by the Creator of NIKE,Phil Knight
4,How to Change Your Mind,6,How to Change Your Mind: What the New Science ...,Michael Pollan
5,Factfulness,5,Factfulness: Ten Reasons We're Wrong About the...,Hans Rosling
6,Man's Search for Meaning,5,Man's Search for Meaning,Viktor E. Frankl
7,Deep Work,5,Deep Work: Rules for Focused Success in a Dist...,Cal Newport
8,Homo Deus,4,Homo Deus: A Brief History of Tomorrow,Yuval Noah Harari
9,The Phoenix Project,4,The Phoenix Project,D.M. Cain


In [187]:
print dfp[dfp['counts']>2].sort_values('counts', ascending=False).reset_index(drop=True).to_html()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>books</th>
      <th>counts</th>
      <th>title</th>
      <th>author</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Bad Blood</td>
      <td>12</td>
      <td>Bad Blood: Secrets and Lies in a Silicon Valle...</td>
      <td>John Carreyrou</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Why We Sleep</td>
      <td>7</td>
      <td>Why We Sleep: Unlocking the Power of Sleep and...</td>
      <td>Matthew Walker</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Magi</td>
      <td>7</td>
      <td>Magi: Uncovering the Secret Society That Read ...</td>
      <td>Adrian Geoffrey Gilbert</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Shoe Dog</td>
      <td>6</td>
      <td>Shoe Dog: A Memoir by the Creator of NIKE</td>
      <td>Phil Knight</td>
    </tr>
    <tr>
      <th>4</th>
      <td>How to Change Your Mind</td>
      <td>6</td>
      <td>How 